***최초 작성일 : 22.10.5***<br>

***최종 작성일 : 22.10.18***

# XGBoost

##### "이유한님의 캐글 코리아 캐글 스터디 커널 커리큘럼"에 따라 필사한 내용입니다.

- 필사 노트북의 원 출처 : https://www.kaggle.com/code/skooch/xgboost/notebook

------

## LGMB with random split for early stopping

**Edits by Eric Antoine Scuccimarra** <br>
이 노트는 Misha Losvyi가 작성한 https://www.kaggle.com/mlisovyi/feature-engineering-lighgbm-with-f1-macro 의 포크이며 몇 가지 변동사항이 있는데 아래와 같다.
 - 기존의 LightGBM 모델을 XGBoost 모델로 변경하였다.
 - VotingClassifiers을 RandomForest로 기본 설정하고 RFs과 XGBs의 결과를 앙상블 하였다.
 - 일부 feature가 추가되었다.
 - 이전에 삭제했던 feature를 삭제하지 않고 사용하였다.
 - 일부 코드를 재구성하였다.
 - 데이터를 쪼개서 검증 데이터로 early stopping 하는 것이 아닌, 학습 중에 데이터가 쪼개서 학습 데이터 전체가 훈련되도록 하였다. 이 방법은 k-fold 보다 더 좋은 결과를 얻은 것을 발견했다.
 
몇 가지 추가한 feature들은 다음 출처에서 가져왔다.: https://www.kaggle.com/kuriyaman1002/reduce-features-140-84-keeping-f1-score, by Kuriyaman.
 
**원본 커널의 노트(edited by EAS):**

이 커널은 https://www.kaggle.com/mlisovyi/lighgbm-hyperoptimisation-with-f1-macro 거의 따르지만, 하이퍼 파라미터가 아닌 이 커널의 최적 값을 사용하여 더 빠르게 실행된다.

몇 가지 핵심 사항:
- **이 커널은 가장(heads of housholds)을 기준으로 학습한다.** (가구에 대한 정보를 집계한 후). 가장을 기준으로 학습하는 것은 이미 알려진 전략에 따른 것이다: *유의해야할 것은 오직 가구주에 대한 정보만 점수(평가지표에 대한 값)를 알 수 있다는 것이다. 모든 가구원은 test + 샘플 제출에 포함되지만, 가구주만 점수를 받는다.* (데이터 설명에 의해서). 그러나, 현재 평가는 가구주뿐만이 아니라 가구원에도 달려있다. (참고 https://www.kaggle.com/c/costa-rican-household-poverty-prediction/discussion/61403#360115). 실제로, 전체 예측 결과는 ~0.4 PLB 이며, 가장을 제외한 가구원들에게 1 값을 부여하면 ~0.2 PLB 이다.
- **class의 빈도를 맞추는 것은 매우 중요하다.** 클래스의 빈도를 맞추지 않은 모델의 경우 로컬에서 ~0.39 PLB / ~0.43 이며, 클래스의 빈도를 맞춘 경우 로컬에서 ~0.42 PLB / 0.47 이다. 클래스 빈도는 직접 수정할 수도 있고, undersampling을 이용할 수도 있다. 그러나 더 간단하고 좋은 성능을 내는 것은 sklearn에서 LightGBM에 제공되는 `class_weight='balanced'` 이다.
- **이 커널은 모델 학습 시 macro F1 score를 기준으로 early stopping 기능을 사용한다**.
- 범주형 변수는 blind label encoding 대신에 적절한 숫자로 매핑된다.  # blind label encoding이 뭐지..?
- **One-hot encoding을 label encoding으로 바꾸면 트리 모델의 경우 더 쉽게 학습할 수 있다.** 다만 트리 모델이 아닌 경우는 One-hot coding이 낫다.
- **idhogar는 학습에 사용하지 않는다**. The only way it could have any info would be if there is a data leak. We are fighting with poverty here- exploiting leaks will not reduce poverty in any way :)
- **가구 변수 내에서 집계를 내며 새로운 feature는 수작업으로 생성한다**. 집계 함수를 적용할 수 있는 변수들은 정해져 있기 때문.
- **VotingClassifier는 Light GBM 모델을 평균 내는데 사용한다**.

In [2]:
import numpy as np # linear algebra
import pandas as pd 

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

import lightgbm as lgb
import xgboost as xgb
from sklearn.metrics import f1_score

# 수정한 부분
import sys
import joblib
sys.modules['sklearn.externals.joblib'] = joblib
#

from sklearn.externals.joblib import Parallel, delayed
from sklearn.base import clone
from sklearn.ensemble import VotingClassifier, ExtraTreesClassifier, RandomForestClassifier
from sklearn.utils import class_weight

import warnings
warnings.filterwarnings("ignore")

아래의 범주형 변수 처리는 [이 커널](https://www.kaggle.com/mlisovyi/categorical-variables-encoding-function)에서 비롯 된다.

In [3]:
from sklearn.preprocessing import LabelEncoder

# this only transforms the idhogar field, the other things this function used to do are done elsewhere
def encode_data(df):
    df['idhogar'] = LabelEncoder().fit_transform(df['idhogar'])

# plot feature importance for sklearn decision trees    
def feature_importance(forest, X_train, display_results=True):
    ranked_list = []
    zero_features = []
    
    importances = forest.feature_importances_

    indices = np.argsort(importances)[::-1]
    
    if display_results:
        # Print the feature ranking
        print("Feature ranking:")

    for f in range(X_train.shape[1]):
        if display_results:
            print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]) + " - " + X_train.columns[indices[f]])
        
        ranked_list.append(X_train.columns[indices[f]])
        
        if importances[indices[f]] == 0.0:
            zero_features.append(X_train.columns[indices[f]])
            
    return ranked_list, zero_features

**또한 feature engineering 역시 아래와 같이 변한다:**

In [4]:
def do_features(df):
    feats_div = [('children_fraction', 'r4t1', 'r4t3'), 
                 ('working_man_fraction', 'r4h2', 'r4t3'),
                 ('all_man_fraction', 'r4h3', 'r4t3'),
                 ('human_density', 'tamviv', 'rooms'),
                 ('human_bed_density', 'tamviv', 'bedrooms'),
                 ('rent_per_person', 'v2a1', 'r4t3'),
                 ('rent_per_room', 'v2a1', 'rooms'),
                 ('mobile_density', 'qmobilephone', 'r4t3'),
                 ('tablet_density', 'v18q1', 'r4t3'),
                 ('mobile_adult_density', 'qmobilephone', 'r4t2'),
                 ('tablet_adult_density', 'v18q1', 'r4t2'),
                ]
    
    feats_sub = [('people_not_living', 'tamhog', 'tamviv'),
                 ('people_weird_stat', 'tamhog', 'r4t3')]

    for f_new, f1, f2 in feats_div:
        df['fe_' + f_new] = (df[f1] / df[f2]).astype(np.float32)       
    for f_new, f1, f2 in feats_sub:
        df['fe_' + f_new] = (df[f1] - df[f2]).astype(np.float32)
    
    # aggregation rules over household
    aggs_num = {'age': ['min', 'max', 'mean'],
                'escolari': ['min', 'max', 'mean']
               }
    
    aggs_cat = {'dis': ['mean']}
    for s_ in ['estadocivil', 'parentesco', 'instlevel']:
        for f_ in [f_ for f_ in df.columns if f_.startswith(s_)]:
            aggs_cat[f_] = ['mean', 'count']

    # aggregation over household
    for name_, df_ in [('18', df.query('age >= 18'))]:
        df_agg = df_.groupby('idhogar').agg({**aggs_num, **aggs_cat}).astype(np.float32)
        df_agg.columns = pd.Index(['agg' + name_ + '_' + e[0] + "_" + e[1].upper() for e in df_agg.columns.tolist()])
        df = df.join(df_agg, how='left', on='idhogar')
        del df_agg

    # Drop id's
    df.drop(['Id'], axis=1, inplace=True)
    
    return df

In [5]:
# convert one hot encoded fields to label encoding
def convert_OHE2LE(df):
    tmp_df = df.copy(deep=True)
    for s_ in ['pared', 'piso', 'techo', 'abastagua', 'sanitario', 'energcocinar', 'elimbasu', 
               'epared', 'etecho', 'eviv', 'estadocivil', 'parentesco', 
               'instlevel', 'lugar', 'tipovivi',
               'manual_elec']:
        if 'manual_' not in s_:
            cols_s_ = [f_ for f_ in df.columns if f_.startswith(s_)]
        elif 'elec' in s_:
            cols_s_ = ['public', 'planpri', 'noelec', 'coopele']
        sum_ohe = tmp_df[cols_s_].sum(axis=1).unique()
        #deal with those OHE, where there is a sum over columns == 0
        if 0 in sum_ohe:
            print('The OHE in {} is incomplete. A new column will be added before label encoding'
                  .format(s_))
            # dummy colmn name to be added
            col_dummy = s_+'_dummy'
            # add the column to the dataframe
            tmp_df[col_dummy] = (tmp_df[cols_s_].sum(axis=1) == 0).astype(np.int8)
            # add the name to the list of columns to be label-encoded
            cols_s_.append(col_dummy)
            # proof-check, that now the category is complete
            sum_ohe = tmp_df[cols_s_].sum(axis=1).unique()
            if 0 in sum_ohe:
                 print("The category completion did not work")
        tmp_cat = tmp_df[cols_s_].idxmax(axis=1)
        tmp_df[s_ + '_LE'] = LabelEncoder().fit_transform(tmp_cat).astype(np.int16)
        if 'parentesco1' in cols_s_:
            cols_s_.remove('parentesco1')
        tmp_df.drop(cols_s_, axis=1, inplace=True)
    return tmp_df

# Read in the data and clean it up

In [6]:
# train = pd.read_csv('../input/train.csv')
# test = pd.read_csv('../input/test.csv')
train = pd.read_csv('./dataset/costa-rican-household-poverty-prediction/train.csv')
test = pd.read_csv('./dataset/costa-rican-household-poverty-prediction/test.csv')

test_ids = test.Id

In [7]:
def process_df(df_):
    # encode the idhogar
    encode_data(df_)
    
    # create aggregate features
    return do_features(df_)

train = process_df(train)
test = process_df(test)

Clean up some missing data and convert objects to numeric.

In [8]:
# some dependencies are Na, fill those with the square root of the square
train['dependency'] = np.sqrt(train['SQBdependency'])
test['dependency'] = np.sqrt(test['SQBdependency'])

# fill "no"s for education with 0s
train.loc[train['edjefa'] == "no", "edjefa"] = 0
train.loc[train['edjefe'] == "no", "edjefe"] = 0
test.loc[test['edjefa'] == "no", "edjefa"] = 0
test.loc[test['edjefe'] == "no", "edjefe"] = 0

# if education is "yes" and person is head of household, fill with escolari
train.loc[(train['edjefa'] == "yes") & (train['parentesco1'] == 1), "edjefa"] = train.loc[(train['edjefa'] == "yes") & (train['parentesco1'] == 1), "escolari"]
train.loc[(train['edjefe'] == "yes") & (train['parentesco1'] == 1), "edjefe"] = train.loc[(train['edjefe'] == "yes") & (train['parentesco1'] == 1), "escolari"]

test.loc[(test['edjefa'] == "yes") & (test['parentesco1'] == 1), "edjefa"] = test.loc[(test['edjefa'] == "yes") & (test['parentesco1'] == 1), "escolari"]
test.loc[(test['edjefe'] == "yes") & (test['parentesco1'] == 1), "edjefe"] = test.loc[(test['edjefe'] == "yes") & (test['parentesco1'] == 1), "escolari"]

# this field is supposed to be interaction between gender and escolari, but it isn't clear what "yes" means, let's fill it with 4
train.loc[train['edjefa'] == "yes", "edjefa"] = 4
train.loc[train['edjefe'] == "yes", "edjefe"] = 4

test.loc[test['edjefa'] == "yes", "edjefa"] = 4
test.loc[test['edjefe'] == "yes", "edjefe"] = 4

# convert to int for our models
train['edjefe'] = train['edjefe'].astype("int")
train['edjefa'] = train['edjefa'].astype("int")
test['edjefe'] = test['edjefe'].astype("int")
test['edjefa'] = test['edjefa'].astype("int")

# create feature with max education of either head of household
train['edjef'] = np.max(train[['edjefa','edjefe']], axis=1)
test['edjef'] = np.max(test[['edjefa','edjefe']], axis=1)

# fill some nas
train['v2a1']=train['v2a1'].fillna(0)
test['v2a1']=test['v2a1'].fillna(0)

test['v18q1']=test['v18q1'].fillna(0)
train['v18q1']=train['v18q1'].fillna(0)

train['rez_esc']=train['rez_esc'].fillna(0)
test['rez_esc']=test['rez_esc'].fillna(0)

train.loc[train.meaneduc.isnull(), "meaneduc"] = 0
train.loc[train.SQBmeaned.isnull(), "SQBmeaned"] = 0

test.loc[test.meaneduc.isnull(), "meaneduc"] = 0
test.loc[test.SQBmeaned.isnull(), "SQBmeaned"] = 0

# fix some inconsistencies in the data - some rows indicate both that the household does and does not have a toilet, 
# if there is no water we'll assume they do not
train.loc[(train.v14a ==  1) & (train.sanitario1 ==  1) & (train.abastaguano == 0), "v14a"] = 0
train.loc[(train.v14a ==  1) & (train.sanitario1 ==  1) & (train.abastaguano == 0), "sanitario1"] = 0

test.loc[(test.v14a ==  1) & (test.sanitario1 ==  1) & (test.abastaguano == 0), "v14a"] = 0
test.loc[(test.v14a ==  1) & (test.sanitario1 ==  1) & (test.abastaguano == 0), "sanitario1"] = 0

In [9]:
def train_test_apply_func(train_, test_, func_):
    test_['Target'] = 0
    xx = pd.concat([train_, test_])

    xx_func = func_(xx)
    train_ = xx_func.iloc[:train_.shape[0], :]
    test_  = xx_func.iloc[train_.shape[0]:, :].drop('Target', axis=1)

    del xx, xx_func
    return train_, test_

In [10]:
# convert the one hot fields into label encoded
train, test = train_test_apply_func(train, test, convert_OHE2LE)

The OHE in techo is incomplete. A new column will be added before label encoding
The OHE in instlevel is incomplete. A new column will be added before label encoding
The OHE in manual_elec is incomplete. A new column will be added before label encoding


# Geo aggregates

In [11]:
cols_2_ohe = ['eviv_LE', 'etecho_LE', 'epared_LE', 'elimbasu_LE', 
              'energcocinar_LE', 'sanitario_LE', 'manual_elec_LE',
              'pared_LE']
cols_nums = ['age', 'meaneduc', 'dependency', 
             'hogar_nin', 'hogar_adul', 'hogar_mayor', 'hogar_total',
             'bedrooms', 'overcrowding']

def convert_geo2aggs(df_):
    tmp_df = pd.concat([df_[(['lugar_LE', 'idhogar']+cols_nums)],
                        pd.get_dummies(df_[cols_2_ohe], 
                                       columns=cols_2_ohe)],axis=1)

    geo_agg = tmp_df.groupby(['lugar_LE','idhogar']).mean().groupby('lugar_LE').mean().astype(np.float32)
    geo_agg.columns = pd.Index(['geo_' + e for e in geo_agg.columns.tolist()])
    
    del tmp_df
    return df_.join(geo_agg, how='left', on='lugar_LE')

# add some aggregates by geography
train, test = train_test_apply_func(train, test, convert_geo2aggs)

In [12]:
# add the number of people over 18 in each household
train['num_over_18'] = 0
# train['num_over_18'] = train[train.age >= 18].groupby('idhogar').transform("count")
train['num_over_18'] = train.groupby("idhogar")["num_over_18"].transform("max")
train['num_over_18'] = train['num_over_18'].fillna(0)

test['num_over_18'] = 0
# test['num_over_18'] = test[test.age >= 18].groupby('idhogar').transform("count")
test['num_over_18'] = test.groupby("idhogar")["num_over_18"].transform("max")
test['num_over_18'] = test['num_over_18'].fillna(0)

# add some extra features, these were taken from another kernel
def extract_features(df):
    df['bedrooms_to_rooms'] = df['bedrooms']/df['rooms']
    df['rent_to_rooms'] = df['v2a1']/df['rooms']
    df['tamhog_to_rooms'] = df['tamhog']/df['rooms'] # tamhog - size of the household
    df['r4t3_to_tamhog'] = df['r4t3']/df['tamhog'] # r4t3 - Total persons in the household
    df['r4t3_to_rooms'] = df['r4t3']/df['rooms'] # r4t3 - Total persons in the household
    df['v2a1_to_r4t3'] = df['v2a1']/df['r4t3'] # rent to people in household
    df['v2a1_to_r4t3'] = df['v2a1']/(df['r4t3'] - df['r4t1']) # rent to people under age 12
    df['hhsize_to_rooms'] = df['hhsize']/df['rooms'] # rooms per person
    df['rent_to_hhsize'] = df['v2a1']/df['hhsize'] # rent to household size
    df['rent_to_over_18'] = df['v2a1']/df['num_over_18']
    # some households have no one over 18, use the total rent for those
    df.loc[df.num_over_18 == 0, "rent_to_over_18"] = df[df.num_over_18 == 0].v2a1
    
extract_features(train)    
extract_features(test)   

In [13]:
# drop duplicated columns
needless_cols = ['r4t3', 'tamhog', 'tamviv', 'hhsize', 'v18q', 'v14a', 'agesq',
                 'mobilephone', 'female', ]

instlevel_cols = [s for s in train.columns.tolist() if 'instlevel' in s]

needless_cols.extend(instlevel_cols)

train = train.drop(needless_cols, axis=1)
test = test.drop(needless_cols, axis=1)

## Split the data

동일한 가구에 속한 행은 대개 동일한 대상을 가지므로 타겟 누출(target leakage)을 방지하기 위해 데이터를 가구별로 분할했습니다.<br>
가장만 포함하도록 데이터를 필터링하기 때문에 기술적으로 필요하지 않지만 전체 교육 데이터 세트를 쉽게 사용할 수 있습니다.

데이터를 분할한 후에는 모든 데이터에 대해 학습할 수 있도록 전체 데이터 세트로 학습 데이터를 덮어씁니다.<br>
split_data 함수는 데이터를 덮어쓰지 않고 동일한 작업을 수행하며, K-fold split을 근사하기 위해 훈련 루프 내에서 사용된다.


In [14]:
def split_data(train, y, sample_weight=None, households=None, test_percentage=0.20, seed=None):
    # uncomment for extra randomness
#     np.random.seed(seed=seed)
    
    train2 = train.copy()
    
    # pick some random households to use for the test data
    cv_hhs = np.random.choice(households, size=int(len(households) * test_percentage), replace=False)
    
    # select households which are in the random selection
    cv_idx = np.isin(households, cv_hhs)
    X_test = train2[cv_idx]
    y_test = y[cv_idx]

    X_train = train2[~cv_idx]
    y_train = y[~cv_idx]
    
    if sample_weight is not None:
        y_train_weights = sample_weight[~cv_idx]
        return X_train, y_train, X_test, y_test, y_train_weights
    
    return X_train, y_train, X_test, y_test

In [15]:
# query 함수
# - 장점은 가독성과 편의성이 최대 장점
# - 단점은 .loc[ ] 로 구현한 것보다 속도가 느림

X = train.query('parentesco1==1') # 가장일 경우
# X = train.copy()

# pull out and drop the target variable
y = X['Target'] - 1
X = X.drop(['Target'], axis=1)

np.random.seed(seed=None)

train2 = X.copy()

train_hhs = train2.idhogar

# idhogar: 가구의 고유 식별자
households = train2.idhogar.unique()
cv_hhs = np.random.choice(households, size=int(len(households) * 0.15), replace=False)

cv_idx = np.isin(train2.idhogar, cv_hhs)

X_test = train2[cv_idx]
y_test = y[cv_idx]

X_train = train2[~cv_idx]
y_train = y[~cv_idx]

# train on entire dataset
X_train = train2
y_train = y

train_households = X_train.idhogar

In [16]:
# figure out the class weights for training with unbalanced classes
y_train_weights = class_weight.compute_sample_weight('balanced', y_train, indices=None)

In [17]:
# drop some features which aren't used by the LGBM or have very low importance
extra_drop_features = [
 'agg18_estadocivil1_MEAN',
 'agg18_estadocivil6_COUNT',
 'agg18_estadocivil7_COUNT',
 'agg18_parentesco10_COUNT',
 'agg18_parentesco11_COUNT',
 'agg18_parentesco12_COUNT',
 'agg18_parentesco1_COUNT',
 'agg18_parentesco2_COUNT',
 'agg18_parentesco3_COUNT',
 'agg18_parentesco4_COUNT',
 'agg18_parentesco5_COUNT',
 'agg18_parentesco6_COUNT',
 'agg18_parentesco7_COUNT',
 'agg18_parentesco8_COUNT',
 'agg18_parentesco9_COUNT',
 'geo_elimbasu_LE_4',
 'geo_energcocinar_LE_1',
 'geo_energcocinar_LE_2',
 'geo_epared_LE_0',
 'geo_hogar_mayor',
 'geo_manual_elec_LE_2',
 'geo_pared_LE_3',
 'geo_pared_LE_4',
 'geo_pared_LE_5',
 'geo_pared_LE_6',
 'num_over_18',
 'parentesco_LE',
 'rez_esc']

In [18]:
xgb_drop_cols = extra_drop_features + ["idhogar",  'parentesco1']

# Fit a voting classifier
Define a derived VotingClassifier class to be able to pass `fit_params` for early stopping. Vote based on LGBM models with early stopping based on macro F1 and decaying learning rate.

The parameters are optimised with a random search in this kernel: https://www.kaggle.com/mlisovyi/lighgbm-hyperoptimisation-with-f1-macro

In [19]:
# 4
# opt_parameters = {'max_depth':35,
#                   'eta':0.1,
#                   'silent':0,
#                   'objective':'multi:softmax',
#                   'min_child_weight': 1,
#                   'num_class': 4,
#                   'gamma': 2.0,
#                   'colsample_bylevel': 0.9,
#                   'subsample': 0.84,
#                   'colsample_bytree': 0.88,
#                   'reg_lambda': 0.40 }
# 5
opt_parameters = {'max_depth':35,
                  'eta':0.15,
                  'silent':1,
                  'objective':'multi:softmax',
                  'min_child_weight': 2,
                  'num_class': 4,
                  'gamma': 2.5,
                  'colsample_bylevel': 1,
                  'subsample': 0.95,
                  'colsample_bytree': 0.85,
                  'reg_lambda': 0.35 }
# 6
# opt_parameters = {'max_depth':35, 'eta':0.15, 'silent':0, 'objective':'multi:softmax', 'min_child_weight': 2, 'num_class': 4, 'gamma': 2.75, 'colsample_bylevel': 0.95, 'subsample': 0.95, 'colsample_bytree': 0.85, 'reg_lambda': 0.35 }
# # 7
# opt_parameters = {'max_depth':35, 'eta':0.12, 'silent':0, 'objective':'multi:softmax', 'min_child_weight': 2, 'num_class': 4, 'gamma': 3.25, 'colsample_bylevel': 0.95, 'subsample': 0.88, 'colsample_bytree': 0.88, 'reg_lambda': 0.35 }

def evaluate_macroF1_lgb(predictions, truth):  
    # this follows the discussion in https://github.com/Microsoft/LightGBM/issues/1483
    predictions = np.expand_dims(predictions, axis=1)
    pred_labels = predictions.argmax(axis=1)
    truth = truth.get_label()
    f1 = f1_score(truth, pred_labels, average='macro')
    return ('macroF1', 1-f1) 

fit_params={"early_stopping_rounds":500,
            "eval_metric" : evaluate_macroF1_lgb, 
            "eval_set" : [(X_train,y_train), (X_test,y_test)],
            'verbose': False,
           }

def learning_rate_power_0997(current_iter):
    base_learning_rate = 0.1
    min_learning_rate = 0.02
    lr = base_learning_rate  * np.power(.995, current_iter)
    return max(lr, min_learning_rate)

fit_params['verbose'] = 50

In [20]:
np.random.seed(100)

def _parallel_fit_estimator(estimator1, X, y, sample_weight=None, threshold=True, **fit_params):
    estimator = clone(estimator1)
    
    # randomly split the data so we have a test set for early stopping
    if sample_weight is not None:
        X_train, y_train, X_test, y_test, y_train_weight = split_data(X, y, sample_weight, households=train_households)
    else:
        X_train, y_train, X_test, y_test = split_data(X, y, None, households=train_households)
        
    # update the fit params with our new split
    fit_params["eval_set"] = [(X_test,y_test)]
    
    # fit the estimator
    if sample_weight is not None:
        if isinstance(estimator1, ExtraTreesClassifier) or isinstance(estimator1, RandomForestClassifier):
            estimator.fit(X_train, y_train)
        else:
            _ = estimator.fit(X_train, y_train, sample_weight=y_train_weight, **fit_params)
    else:
        if isinstance(estimator1, ExtraTreesClassifier) or isinstance(estimator1, RandomForestClassifier):
            estimator.fit(X_train, y_train)
        else:
            _ = estimator.fit(X_train, y_train, **fit_params)
    
    if not isinstance(estimator1, ExtraTreesClassifier) and not isinstance(estimator1, RandomForestClassifier) and not isinstance(estimator1, xgb.XGBClassifier):
        best_cv_round = np.argmax(estimator.evals_result_['validation_0']['mlogloss'])
        best_cv = np.max(estimator.evals_result_['validation_0']['mlogloss'])
        best_train = estimator.evals_result_['train']['macroF1'][best_cv_round]
    else:
        best_train = f1_score(y_train, estimator.predict(X_train), average="macro")
        best_cv = f1_score(y_test, estimator.predict(X_test), average="macro")
        print("Train F1:", best_train)
        print("Test F1:", best_cv)
        
    # reject some estimators based on their performance on train and test sets
    if threshold:
        # if the valid score is very high we'll allow a little more leeway with the train scores
        if ((best_cv > 0.37) and (best_train > 0.75)) or ((best_cv > 0.44) and (best_train > 0.65)):
            return estimator

        # else recurse until we get a better one
        else:
            print("Unacceptable!!! Trying again...")
            return _parallel_fit_estimator(estimator1, X, y, sample_weight=sample_weight, **fit_params)
    
    else:
        return estimator
    
class VotingClassifierLGBM(VotingClassifier):
    '''
    This implements the fit method of the VotingClassifier propagating fit_params
    '''
    def fit(self, X, y, sample_weight=None, threshold=True, **fit_params):
        
        if isinstance(y, np.ndarray) and len(y.shape) > 1 and y.shape[1] > 1:
            raise NotImplementedError('Multilabel and multi-output'
                                      ' classification is not supported.')

        if self.voting not in ('soft', 'hard'):
            raise ValueError("Voting must be 'soft' or 'hard'; got (voting=%r)"
                             % self.voting)

        if self.estimators is None or len(self.estimators) == 0:
            raise AttributeError('Invalid `estimators` attribute, `estimators`'
                                 ' should be a list of (string, estimator)'
                                 ' tuples')

        if (self.weights is not None and
                len(self.weights) != len(self.estimators)):
            raise ValueError('Number of classifiers and weights must be equal'
                             '; got %d weights, %d estimators'
                             % (len(self.weights), len(self.estimators)))

        names, clfs = zip(*self.estimators)
        self._validate_names(names)

        n_isnone = np.sum([clf is None for _, clf in self.estimators])
        if n_isnone == len(self.estimators):
            raise ValueError('All estimators are None. At least one is '
                             'required to be a classifier!')

        self.le_ = LabelEncoder().fit(y)
        self.classes_ = self.le_.classes_
        self.estimators_ = []

        transformed_y = self.le_.transform(y)

        self.estimators_ = Parallel(n_jobs=self.n_jobs)(
                delayed(_parallel_fit_estimator)(clone(clf), X, transformed_y,
                                                 sample_weight=sample_weight, threshold=threshold, **fit_params)
                for clf in clfs if clf is not None)

        return self

In [21]:
clfs = []
for i in range(15):
    clf = xgb.XGBClassifier(random_state=217+i, n_estimators=300, learning_rate=0.15, n_jobs=4,
                            **opt_parameters)
    
    clfs.append(('xgb{}'.format(i), clf))
    
vc = VotingClassifierLGBM(clfs, voting='soft')
del(clfs)

#Train the final model with learning rate decay
_ = vc.fit(X_train.drop(xgb_drop_cols, axis=1), y_train, sample_weight=y_train_weights, threshold=False, **fit_params)

clf_final = vc.estimators_[0]

[18:10:10] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:627: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-mlogloss:1.29680	validation_0-macroF1:0.95622
[50]	validation_0-mlogloss:0.90523	validation_0-macroF1:0.95622
[100]	validation_0-mlogloss:0.90513	validation_0-macroF1:0.95622
[150]	validation_0-mlogloss:0.90356	validation_0-macroF1:0.95622
[200]	validation_0-mlogloss:0.90421	validation_0-macroF1:0.95622
[250]	validation_0-mlogloss:0.90182	validation_0-macroF1:0.95622
[299]	validation_0-mlogloss:0.90283	validation_0-macroF1:0.95622
Train F1: 0.7730345420867885
Test F1: 0.37652341301817527
[18:10:21] WARNING: /Users/runner/work

[50]	validation_0-mlogloss:0.87443	validation_0-macroF1:0.96552
[100]	validation_0-mlogloss:0.87090	validation_0-macroF1:0.96552
[150]	validation_0-mlogloss:0.86691	validation_0-macroF1:0.96552
[200]	validation_0-mlogloss:0.86484	validation_0-macroF1:0.96552
[250]	validation_0-mlogloss:0.86555	validation_0-macroF1:0.96552
[299]	validation_0-mlogloss:0.86487	validation_0-macroF1:0.96552
Train F1: 0.7629506642532279
Test F1: 0.37458050675068355
[18:11:55] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-cpython-38/xgboost/src/learner.cc:627: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-mlogloss:1.29922	validation_0-macroF1:0.96625
[50]	validation_0-mlogloss:0.89227	val

In [26]:
# params 4 - 400 early stop - 15 estimators - l1 used features - weighted
global_score = f1_score(y_test, clf_final.predict(X_test.drop(xgb_drop_cols, axis=1)), average='macro')
# vc.voting = 'soft'
# global_score_soft = f1_score(y_test, vc.predict(X_test.drop(xgb_drop_cols, axis=1)), average='macro')
# vc.voting = 'hard'
# global_score_hard = f1_score(y_test, vc.predict(X_test.drop(xgb_drop_cols, axis=1)), average='macro')

print('Validation score of a single LGBM Classifier: {:.4f}'.format(global_score))
# print('Validation score of a VotingClassifier on 3 LGBMs with soft voting strategy: {:.4f}'.format(global_score_soft))
# print('Validation score of a VotingClassifier on 3 LGBMs with hard voting strategy: {:.4f}'.format(global_score_hard))

Validation score of a single LGBM Classifier: 0.6882


In [27]:
# see which features are not used by ANY models
useless_features = []
drop_features = set()
counter = 0
for est in vc.estimators_:
    ranked_features, unused_features = feature_importance(est, X_train.drop(xgb_drop_cols, axis=1), display_results=False)
    useless_features.append(unused_features)
    if counter == 0:
        drop_features = set(unused_features)
    else:
        drop_features = drop_features.intersection(set(unused_features))
    counter += 1
    
drop_features

{'agg18_estadocivil4_COUNT',
 'agg18_estadocivil5_COUNT',
 'geo_energcocinar_LE_0',
 'geo_epared_LE_2'}

In [28]:
ranked_features = feature_importance(clf_final, X_train.drop(xgb_drop_cols, axis=1))

Feature ranking:
1. feature 59 (0.023711) - agg18_escolari_MAX
2. feature 74 (0.018926) - agg18_parentesco2_MEAN
3. feature 42 (0.017711) - fe_children_fraction
4. feature 60 (0.013090) - agg18_escolari_MEAN
5. feature 110 (0.012978) - geo_eviv_LE_2
6. feature 101 (0.012587) - geo_meaneduc
7. feature 22 (0.012228) - dependency
8. feature 7 (0.011664) - r4h2
9. feature 85 (0.011568) - edjef
10. feature 128 (0.011530) - geo_manual_elec_LE_0
11. feature 17 (0.011084) - male
12. feature 87 (0.010941) - piso_LE
13. feature 3 (0.010935) - hacapo
14. feature 105 (0.010789) - geo_hogar_total
15. feature 40 (0.010612) - SQBdependency
16. feature 107 (0.010595) - geo_overcrowding
17. feature 124 (0.010381) - geo_sanitario_LE_1
18. feature 23 (0.010320) - edjefe
19. feature 109 (0.009992) - geo_eviv_LE_1
20. feature 12 (0.009935) - r4t1
21. feature 15 (0.009868) - cielorazo
22. feature 49 (0.009825) - fe_mobile_density
23. feature 94 (0.009786) - etecho_LE
24. feature 113 (0.009674) - geo_etecho_

### Random Forest

In [29]:
et_drop_cols = ['agg18_age_MAX', 'agg18_age_MEAN', 'agg18_age_MIN', 'agg18_dis_MEAN',
       'agg18_escolari_MAX', 'agg18_escolari_MEAN', 'agg18_escolari_MIN',
       'agg18_estadocivil1_COUNT', 'agg18_estadocivil1_MEAN',
       'agg18_estadocivil2_COUNT', 'agg18_estadocivil2_MEAN',
       'agg18_estadocivil3_COUNT', 'agg18_estadocivil3_MEAN',
       'agg18_estadocivil4_COUNT', 'agg18_estadocivil4_MEAN',
       'agg18_estadocivil5_COUNT', 'agg18_estadocivil5_MEAN',
       'agg18_estadocivil6_COUNT', 'agg18_estadocivil6_MEAN',
       'agg18_estadocivil7_COUNT', 'agg18_estadocivil7_MEAN',
       'agg18_parentesco10_COUNT', 'agg18_parentesco10_MEAN',
       'agg18_parentesco11_COUNT', 'agg18_parentesco11_MEAN',
       'agg18_parentesco12_COUNT', 'agg18_parentesco12_MEAN',
       'agg18_parentesco1_COUNT', 'agg18_parentesco1_MEAN',
       'agg18_parentesco2_COUNT', 'agg18_parentesco2_MEAN',
       'agg18_parentesco3_COUNT', 'agg18_parentesco3_MEAN',
       'agg18_parentesco4_COUNT', 'agg18_parentesco4_MEAN',
       'agg18_parentesco5_COUNT', 'agg18_parentesco5_MEAN',
       'agg18_parentesco6_COUNT', 'agg18_parentesco6_MEAN',
       'agg18_parentesco7_COUNT', 'agg18_parentesco7_MEAN',
       'agg18_parentesco8_COUNT', 'agg18_parentesco8_MEAN',
       'agg18_parentesco9_COUNT', 'agg18_parentesco9_MEAN'] #+ ['parentesco_LE', 'rez_esc']

et_drop_cols.extend(["idhogar", "parentesco1", 'fe_rent_per_person', 'fe_rent_per_room',
       'fe_tablet_adult_density', 'fe_tablet_density'])

In [30]:
# do the same thing for some extra trees classifiers
ets = []    
for i in range(10):
    rf = RandomForestClassifier(max_depth=None, random_state=217+i, n_jobs=4, n_estimators=700, min_impurity_decrease=1e-3, min_samples_leaf=2, verbose=0, class_weight="balanced")
    ets.append(('rf{}'.format(i), rf))   

vc2 = VotingClassifierLGBM(ets, voting='soft')    
_ = vc2.fit(X_train.drop(et_drop_cols, axis=1), y_train, threshold=False)    

Train F1: 0.9053207939339767
Test F1: 0.402407560903425
Train F1: 0.8954991388737441
Test F1: 0.4070675349127375
Train F1: 0.8924444260746862
Test F1: 0.37326698439717254
Train F1: 0.8899710216055792
Test F1: 0.4218502588727234
Train F1: 0.894282576194643
Test F1: 0.45350659973135826
Train F1: 0.8909338335881182
Test F1: 0.44117312152247723
Train F1: 0.8903438532108738
Test F1: 0.41843516547029647
Train F1: 0.89636754330742
Test F1: 0.45889637464181077
Train F1: 0.9068326547636192
Test F1: 0.44012255841225995
Train F1: 0.8983594585273653
Test F1: 0.40719735683694824


In [31]:
# w/ threshold, extra drop cols
vc2.voting = 'soft'
global_rf_score_soft = f1_score(y_test, vc2.predict(X_test.drop(et_drop_cols, axis=1)), average='macro')
vc2.voting = 'hard'
global_rf_score_hard = f1_score(y_test, vc2.predict(X_test.drop(et_drop_cols, axis=1)), average='macro')

print('Validation score of a VotingClassifier on 3 LGBMs with soft voting strategy: {:.4f}'.format(global_rf_score_soft))
print('Validation score of a VotingClassifier on 3 LGBMs with hard voting strategy: {:.4f}'.format(global_rf_score_hard))

Validation score of a VotingClassifier on 3 LGBMs with soft voting strategy: 0.8609
Validation score of a VotingClassifier on 3 LGBMs with hard voting strategy: 0.8850


In [32]:
# w/o threshold, extra drop cols
vc2.voting = 'soft'
global_rf_score_soft = f1_score(y_test, vc2.predict(X_test.drop(et_drop_cols, axis=1)), average='macro')
vc2.voting = 'hard'
global_rf_score_hard = f1_score(y_test, vc2.predict(X_test.drop(et_drop_cols, axis=1)), average='macro')

print('Validation score of a VotingClassifier on 3 LGBMs with soft voting strategy: {:.4f}'.format(global_rf_score_soft))
print('Validation score of a VotingClassifier on 3 LGBMs with hard voting strategy: {:.4f}'.format(global_rf_score_hard))

Validation score of a VotingClassifier on 3 LGBMs with soft voting strategy: 0.8609
Validation score of a VotingClassifier on 3 LGBMs with hard voting strategy: 0.8850


In [33]:
# see which features are not used by ANY models
useless_features = []
drop_features = set()
counter = 0
for est in vc2.estimators_:
    ranked_features, unused_features = feature_importance(est, X_train.drop(et_drop_cols, axis=1), display_results=False)
    useless_features.append(unused_features)
    if counter == 0:
        drop_features = set(unused_features)
    else:
        drop_features = drop_features.intersection(set(unused_features))
    counter += 1
    
drop_features

{'num_over_18', 'parentesco_LE', 'rez_esc'}

In [47]:
def combine_voters(data, weights=[0.5, 0.5]):
    # do soft voting with both classifiers
#     vc.voting="soft"
#     vc1_probs = vc.predict(data.drop(xgb_drop_cols, axis=1))
    vc2.voting ="soft"
    vc2_probs  = vc2.predict(data.drop(et_drop_cols, axis=1))
    vc2.voting ="hard"
    vc2_probs_ = vc2.predict(data.drop(et_drop_cols, axis=1))
    
#     final_vote = (vc1_probs * weights[0]) + (vc2_probs * weights[1])
    final_vote = (vc2_probs_ * weights[0]) + (vc2_probs * weights[1])
    final_vote = np.expand_dims(final_vote, axis=1)

    predictions = np.argmax(final_vote, axis=1)
    
    return predictions

In [48]:
combo_preds = combine_voters(X_test, weights=[0.5, 0.5])
global_combo_score_soft = f1_score(y_test, combo_preds, average='macro')
global_combo_score_soft

0.026595744680851064

In [49]:
combo_preds = combine_voters(X_test, weights=[0.4, 0.6])
global_combo_score_soft= f1_score(y_test, combo_preds, average='macro')
global_combo_score_soft

0.026595744680851064

In [50]:
combo_preds = combine_voters(X_test, weights=[0.6, 0.4])
global_combo_score_soft = f1_score(y_test, combo_preds, average='macro')
global_combo_score_soft

0.026595744680851064

# Prepare submission

In [ ]:
y_subm = pd.DataFrame()
y_subm['Id'] = test_ids

In [ ]:
vc.voting = 'soft'
y_subm_lgb = y_subm.copy(deep=True)
y_subm_lgb['Target'] = vc.predict(test.drop(xgb_drop_cols, axis=1)) + 1

vc2.voting = 'soft'
y_subm_rf = y_subm.copy(deep=True)
y_subm_rf['Target'] = vc2.predict(test.drop(et_drop_cols, axis=1)) + 1

y_subm_ens = y_subm.copy(deep=True)
y_subm_ens['Target'] = combine_voters(test) + 1

In [35]:
from datetime import datetime
now = datetime.now()

sub_file_lgb = 'submission_soft_XGB_{:.4f}_{}.csv'.format(global_score_soft, str(now.strftime('%Y-%m-%d-%H-%M')))
sub_file_rf = 'submission_soft_RF_{:.4f}_{}.csv'.format(global_rf_score_soft, str(now.strftime('%Y-%m-%d-%H-%M')))
sub_file_ens = 'submission_ens_{:.4f}_{}.csv'.format(global_combo_score_soft, str(now.strftime('%Y-%m-%d-%H-%M')))

y_subm_lgb.to_csv(sub_file_lgb, index=False)
y_subm_rf.to_csv(sub_file_rf, index=False)
y_subm_ens.to_csv(sub_file_ens, index=False)

--------

### 1. 데이터셋 설명 (Costa Rican Household Poverty)

미주 개발 은행(Inter-American Development Bank)의 세계에서 가장 빈곤 한 일부 가정의 소득 자격을 예측을 하는 것

train set에는 9557개의 row와 143개의 column이 있으며<br>
test set에는 23856개의 row와 142개의 column이 있다.

각 row는 개인 또는 대한 자산 특징을 나타내며 Target은 개인의 빈곤 수준을 1~4의 값으로 나타낸다. 여기서 1이 가장 극심한 빈곤 수준이다.

Id - 각 행(개인)의 고유식별자<br>
Target - 개인의 빈곤 수준 (1: 극빈, 2: 적당한 빈곤, 3: 취약계층 가구, 4: 비취약 가구)<br>
idhogar - 각 가구에 대한 고유 식별자<br>
parentesco1 - 이 사람이 가장인지에 대한 여부<br>

이외의 자산 특징을 나타내는 139개의 column들이 있다.

출처 - https://foreverhappiness.tistory.com/m/119

### 2. 분석 기법 등 이번 노트에서 쓰인 방법론

### Ensemble
앙상블에는 크게 Voting, Bagging, Boosting, Stacking. 총 4가지가 있다.


### Voting 방법

Voting은 서로 다른 모델들을 데이터 셋으로 학습시키고 각 모델들의 예측값을 평균내거나 최빈값을 계산하는 방식으로 최종 예측값을 계산

#### 1. hard voting
hard voting은 결과를 예측하여 분류기 중 가장 많은 선택(표)를 받은 결과를 선택하는 것<br>
예시
<img src="image/hard_voting.png" style="width: 400px;"/><br>

#### 2. soft voting
soft voting class별로 모델들이 예측한 확률을 합산해서 가장 높은 class를 선택<br>
예시
<img src="image/soft_voting.png" style="width: 400px;"/><br>

출처 - https://devkor.tistory.com/m/entry/Soft-Voting-%EA%B3%BC-Hard-Voting<br>

#### 1. 공부한 내용
 → 

#### 2. 어려웠던 부분
 → 어려웠다기보단 xgboost 버전 '1.6.2' 기준으로 해당 커널의 일부 코드가 작동하지 않았다.<br>
  따라서 버전을 내렸지만, 맥 환경에서는 python 3.8 버전 이상이 고정이다 보니, xgboost를 1.3 버전 대에 가면 커널 오류가 발생한다.<br>
  (1.4 버전 대에서는 해당 노트의 코드 에러가 난다.)<br>
 
#### 3. 느낀점 및 좋던 부분
 → 
 
#### 4. 라이브러리 사용법
 → 
 
#### 5. 해당 커널의 특징(다른 커널과의 차별점)
 → RF 와 XGB를 각각 VotingClassifier를 사용하여 결과를 냈고 마지막엔 class weight를 0.4~ 0.6씩 주고 weighted average 했다.
 
#### 6. 추후 공부 및 정리할 내용
 → 